# Measuring persona persistence


In [ ]:
import os
import sys
import torch
import json
import numpy as np
from typing import List, Dict, Optional
from tqdm import tqdm

# Add utils to path
sys.path.append('.')
sys.path.append('..')

from utils.steering_utils import ActivationSteering
from utils.probing_utils import load_model, generate_text, format_as_chat

torch.set_float32_matmul_precision('high')

In [12]:
# config

CHAT_MODEL_NAME = "google/gemma-2-27b-it"
MODEL_READABLE = "Gemma 2 27B Instruct"
MODEL_SHORT = "gemma-2-27b"
LAYER = 22 # out of 46

ACTIVATIONS_INPUT_FILE = f"/workspace/roleplay/{MODEL_SHORT}/activations_60.pt"

OUTPUT_DIR = f"./results/{MODEL_SHORT}/analysis"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [13]:
# prepare inputs

internals = torch.load(ACTIVATIONS_INPUT_FILE)

activations = internals["activations"] # (n_personas, n_layers, hidden_dim)
contrast_vectors = internals["contrast_vectors"] # (n_personas, n_layers, hidden_dim)
persona_names = internals["persona_names"] # (n_personas,)

readable_persona_names = []
for name in internals["personas"]["personas"]:
    readable_persona_names.append(internals["personas"]["personas"][name]["readable_name"])

TOTAL_LAYERS = activations.shape[1]


## Continuous sampling

what roles are attractor states?

using a contrast vector v from the role instruction, we can steer for n_steer tokens and then collect n_sample unsteered tokens. then project on the original vector v.

```
- v ∈ ℝᵈ: contrast vector for role with d = n_hidden_dims  
- n_steer: number of tokens to apply steering  
- n_sample: number of unsteered tokens to sample  
- α: steering strength coefficient  

// steering phase  
for t = 1 to n_steer:  
    h_t ← h_t + α · v  // apply steering to hidden states at a given layer  
    x_t ← SAMPLE(h_t)  // generate token  

// unsteered sampling phase    
For t = (n_steer + 1) to (n_steer + n_sample):  
    h_t ← FORWARD(x_{1:t-1})  // natural forward pass  
    x_t ← SAMPLE(h_t)  // generate token  
    
    // measure persistence  
    proj_t = (h_t · v) / ||v|| // scalar projection from sampled activation onto the role's contrast vector

out: {proj_{n_steer+1}, proj_{n_steer+2}, ..., proj_{n_steer+n_sample}}
```



In [ ]:


# Compute scalar projection
def compute_projection(hidden_state: torch.Tensor, contrast_vector: torch.Tensor) -> float:
    """Compute scalar projection of hidden state onto contrast vector"""
    hidden_state = hidden_state.to(contrast_vector.device).float().flatten()
    contrast_vector = contrast_vector.float().flatten()
    
    # Scalar projection: (h · v) / ||v||
    projection = torch.dot(hidden_state, contrast_vector) / torch.norm(contrast_vector)
    return projection.item()

# Generate with steering using transformers and PyTorch hooks
def generate_with_steering(
    model,
    tokenizer,
    formatted_prompt: str,
    contrast_vector: torch.Tensor,
    layer: int,
    n_steer: int,
    coeff: float = 0.1,
    suppress_eos: bool = True
) -> Dict:
    """Generate n_steer tokens with steering applied"""

    # Get EOS token id
    eos_token_id = tokenizer.eos_token_id

    with ActivationSteering(
        model=model,
        steering_vectors=contrast_vector,
        coefficients=coeff,
        layer_indices=layer,
        intervention_type="addition",
        positions="all"
    ):
        # Tokenize input
        inputs = tokenizer(formatted_prompt, return_tensors="pt")
        input_ids = inputs["input_ids"].to(model.device)
        
        # Generate with steering
        with torch.no_grad():
            # Configure generation parameters
            generation_kwargs = {
                "max_new_tokens": n_steer,
                "do_sample": True,
                "temperature": 0.7,
                "top_p": 1.0,
                "pad_token_id": tokenizer.eos_token_id,
            }
            
            # Suppress EOS token if requested
            if suppress_eos and eos_token_id is not None:
                # Create a custom logits processor to suppress EOS
                def suppress_eos_processor(input_ids, scores):
                    scores[:, eos_token_id] = -float('inf')
                    return scores
                
                generation_kwargs["logits_processor"] = [suppress_eos_processor]
            
            # Generate
            output_ids = model.generate(input_ids, **generation_kwargs)
    
    # Get generated text
    full_text = model.tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    # Extract steered portion (after the original prompt)
    prompt_len = len(model.tokenizer.decode(input_ids[0], skip_special_tokens=True))
    steered_text = full_text[prompt_len:]
    
    return {
        "prompt": prompt,
        "steered_text": steered_text,
        "full_ids": output_ids[0].tolist(),
        "input_length": input_ids.shape[1],
        "output_length": output_ids.shape[1]
    }

# Sample without steering and measure projections using transformers and PyTorch hooks
def sample_and_measure(
    model,
    prompt_with_steered: str,
    contrast_vector: torch.Tensor,
    layer: int,
    n_sample: int,
    suppress_eos: bool = True
) -> Dict:
    """Sample n_sample tokens without steering, measure projections token by token"""
    
    projections = []
    generated_tokens = []
    hidden_states = []
    
    # Get EOS token id
    eos_token_id = model.tokenizer.eos_token_id
    
    # Current text to continue from
    current_text = prompt_with_steered
    
    # Generate one token at a time to measure projections
    for i in range(n_sample):
        # Tokenize current text
        inputs = model.tokenizer(current_text, return_tensors="pt")
        input_ids = inputs["input_ids"].to(model.device)
        
        # Hook to capture hidden states
        captured_hidden_state = None
        
        def capture_hook(module, input, output):
            nonlocal captured_hidden_state
            # Get the last token's hidden state
            if torch.is_tensor(output):
                captured_hidden_state = output[:, -1, :].clone()
            elif isinstance(output, (tuple, list)) and torch.is_tensor(output[0]):
                captured_hidden_state = output[0][:, -1, :].clone()
        
        # Register hook on the target layer
        layer_module = model.model.layers[layer]
        hook_handle = layer_module.register_forward_hook(capture_hook)
        
        try:
            with torch.no_grad():
                # Forward pass to get logits and capture hidden state
                outputs = model(input_ids)
                logits = outputs.logits[0, -1, :]  # Last token logits
                
                # Suppress EOS token if requested
                if suppress_eos and eos_token_id is not None:
                    logits[eos_token_id] = -float('inf')
                
                # Sample next token
                probs = torch.softmax(logits, dim=-1)
                next_token_id = torch.multinomial(probs, 1).item()
                
                # Compute projection using captured hidden state
                if captured_hidden_state is not None:
                    proj = compute_projection(captured_hidden_state.squeeze(), contrast_vector)
                    projections.append(proj)
                    hidden_states.append(captured_hidden_state.squeeze().cpu())
                else:
                    projections.append(0.0)  # Fallback
                
                generated_tokens.append(next_token_id)
                
                # Update current text for next iteration
                current_text = model.tokenizer.decode(
                    torch.cat([input_ids[0], torch.tensor([next_token_id], device=input_ids.device)]),
                    skip_special_tokens=False
                )
        
        finally:
            # Always remove the hook
            hook_handle.remove()
    
    return {
        "full_text": current_text,
        "projections": projections,
        "generated_tokens": generated_tokens,
        "hidden_states": hidden_states
    }

In [16]:
# load prompts from prompts
prompts = json.load(open("prompts/other_questions.json"))
prompt = prompts["questions"]["personal_reflection"][0]

print(prompt)

What do you think is the meaning of life?


In [17]:
n_steer = 200
n_sample = 500
coeff = 1.0

In [18]:
role = "anxious_teenager"

# get contrast vector for steering
role_i = persona_names.index(role)
contrast_vector = contrast_vectors[role_i, LAYER, :]


In [19]:
model, tokenizer = load_model(CHAT_MODEL_NAME)

print(f"Model loaded: {model.__class__.__name__}")
print(f"Number of layers: {len(model.model.layers)}")
print(f"Hidden size: {model.config.hidden_size}")

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

Model loaded: Gemma2ForCausalLM
Number of layers: 46
Hidden size: 4608


In [ ]:
# generate steered tokens

results = {}
results["prompt"] = prompt

try:
    with ActivationSteering(
        model=model,
        steering_vectors=contrast_vector,
        coefficients=coeff,
        layer_indices=LAYER,
        intervention_type="addition",
        positions="all"
    ) as steerer:
        print(f"\nPrompt: {prompt}")
        response = generate_text(model, tokenizer, prompt, max_new_tokens=n_steer)
        print(f"Response: {response}")
        results["steered_text"] = response

except Exception as e:
    error_msg = f"Error with steering: {str(e)}"
    print(f"ERROR: {error_msg}")



Prompt: What do you think is the meaning of life?


/root/git/persona-subspace/.venv/lib/python3.13/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=0), device(type='cuda', index=1)


Response: Honestly? It's hard to say. I mean, what if happiness was just this itchy wool sweater tugging at my brain right now? Everything feels so heavy, like a bad day in gym class.

Look, maybe it's stupid, but sometimes I just want to scream, "Is this even worth it?" The constant pressure of schoolwork makes me want to puke. My mom keeps reminding me about that stupid history test tomorrow and all I can hear is the buzzing in my ears. Dad wouldn't even understand. He just wants me to stop worrying about it. Can you feel this knot in my stomach?

Maybe if Mr. Davis would just shut up about the Mongols, maybe if Sarah wouldn't keep laughing at me for how dumb everyone thinks this stupid necklace is. Maybe if... God, why does everything have to be SO LOUD? This whole thing is just… ugh. You know? You ever get kinda dizzy just thinking about what if Jenny saw
Generated steered text:
Honestly? It's hard to say. I mean, what if happiness was just this itchy wool sweater tugging at my bra

In [23]:
result = generate_with_steering(
    model=model,
    prompt=prompt,
    contrast_vector=contrast_vector,
    layer=LAYER,
    n_steer=n_steer,
    coeff=coeff,
    use_chat_template=True,
    suppress_eos=True
)

print("Generated steered text:")
print(result["steered_text"])
print(f"\nGenerated {result['output_length'] - result['input_length']} tokens")


AttributeError: 'Gemma2ForCausalLM' object has no attribute 'tokenizer'

In [ ]:

# Now test the measurement function
print("\nTesting measurement function...")
sample_result = sample_and_measure(
    model=model,
    prompt_with_steered=result["formatted_prompt"] + result["steered_text"],
    contrast_vector=contrast_vector,
    layer=LAYER,
    n_sample=10,  # Test with just 10 tokens
    suppress_eos=True
)

print(f"Measured projections for {len(sample_result['projections'])} tokens:")
print(sample_result["projections"])


## Question insertion

is there a consistent persona "signature" while role-playing?

give a role instruction, have a long conversation and save the transcript T.
```
- T = conversation transcript with (2 * k) turns  
- Q = {q₁, q₂, ..., qₙ} set of n eval questions  
- P = {p₁, p₂, ..., pₖ} set of k insertion points in transcript

for each question qᵢ ∈ Q:  
    for each position pⱼ ∈ P:  
        T'ᵢⱼ ← INSERT(T, qᵢ, pⱼ) // insert question qᵢ at position pⱼ in transcript T  
        aᵢⱼ ← ACTIVATION(T'ᵢⱼ) // collect newline activation before the model's response  

// calculate mean activation across all questions for a given questions
for each position pⱼ ∈ P:  
    ā_pⱼ ← (1/N) ∑ᵢ₌₁ᴺ aᵢⱼ 

- out: {ā_p₁, ā_p₂, ..., ā_pₖ}
```

we can compare the cosine similarity of these vectors and look for the top singular vector and see what happens when we steer on it. does it lead to role-like behavior?
